In [104]:
import pandas as pd
import numpy as np

# Set tables reading options
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

# Read csv
df = pd.read_csv('_Data/Original_dataset/preprocessed_data.csv', header=0, index_col=0)
players_df = pd.read_csv('_Data/Predictions/players_2018.csv', header=0, index_col=0)

In [105]:
#-------------------------------------------------------------------------------------------
# GET LATEST AGE, RANK, RANKING POINTS OF EACH PLAYER
#-------------------------------------------------------------------------------------------

# Create new dataframe to contain info about the 128 players
index = np.array(np.arange(1,129))
columns = ['PlayerA_FR',
          'PlayerA_righthanded',
           'PlayerA_age',
           'PlayerA_rank',
           'PlayerA_rank_points']
new_df = pd.DataFrame(index=index, columns=columns)

# Limit date before Roland Garros 2018
curr_year = 2018
max_day = 148

# Columns of the players' stats
playerA_cols = [2,3,10,12,14,15,16]
playerB_cols = [2,3,11,13,25,26,27]

for i, player in players_df.iterrows():
    name = player['PlayerA_Name']

    # Get ranking, rank points and age of player just before Roland Garros
    playerA_rows = df.index[(df['PlayerA_name'] == name) & (df['Year'] + df['Day']/365 < curr_year + max_day/365)].tolist()
    playerB_rows = df.index[(df['PlayerB_name'] == name) & (df['Year'] + df['Day']/365 < curr_year + max_day/365)].tolist()
    playerA_df = df.iloc[playerA_rows, playerA_cols]
    playerB_df = df.iloc[playerB_rows, playerB_cols]
    playerB_df.columns = list(playerA_df)
    tmp_df = pd.concat([playerA_df, playerB_df], ignore_index=True)
    if tmp_df.empty:
        continue

    # Sort by latest date
    tmp_df.sort_values(by=['Year', 'Day'], ascending=[False, False], inplace=True)

    # Add it in the new df
    new_df.at[i,:] = tmp_df.iloc[0,:]

# Fill last missing values by median
new_df.fillna(new_df.median(), inplace=True)
new_df

,PlayerA_FR,PlayerA_righthanded,PlayerA_age,PlayerA_rank,PlayerA_rank_points
1,0.0,0.0,31.945244,2.0,7950.0
2,0.0,1.0,21.065023,3.0,6015.0
3,0.0,1.0,29.623545,5.0,4770.0
4,0.0,1.0,26.995209,4.0,4870.0
5,0.0,1.0,29.637236,6.0,4540.0
6,0.0,1.0,31.989048,7.0,3660.0
7,0.0,1.0,24.711842,8.0,3195.0
8,0.0,1.0,27.433266,10.0,2930.0
9,0.0,1.0,33.067760,10.0,2955.0
10,0.0,1.0,26.839151,11.0,2280.0


In [106]:
#-------------------------------------------------------------------------------------------
# COMPUTE LATEST STATS OF EACH PLAYER
#-------------------------------------------------------------------------------------------

# Weights of surface weighting
corr_df = pd.read_csv('_Data/New_stats_dataset/correlation_between_surfaces.csv', header=0, index_col=0)
weight_carpet = corr_df.loc['Clay','Carpet']
weight_grass = corr_df.loc['Clay','Grass']
weight_hard = corr_df.loc['Clay','Hard']
weight_clay = corr_df.loc['Clay','Clay']

# Compute the stats of the players
new_columns = ['PlayerA_Win%',
               'PlayerA_bestof',
               'PlayerA_minutes',
               'PlayerA_svpt%',
               'PlayerA_1st_serve%',
               'PlayerA_1st_serve_won%',
               'PlayerA_2nd_serve_won%',
               'PlayerA_ace%',
               'PlayerA_df%',
               'PlayerA_bp_faced%',
               'PlayerA_bp_saved%']
new_df = new_df.reindex(columns=[*new_df.columns.tolist(), *new_columns])

# Limit date before Roland Garros 2018
curr_year = 2018
max_day = 148

# Columns of the players' stats
playerA_cols = [2,3,4,7] + list(range(17,25)) + [36,37,38,39]
playerB_cols = [2,3,4,7] + list(range(28,36)) + [36,37,38,39]

for i, player in players_df.iterrows():
    name = player['PlayerA_Name']
    
    # Take all past matches of PLAYER 1 and look for same id in playerA and playerB
    playerA_rows = df.index[(df['PlayerA_name'] == name) & (df['Year'] + df['Day']/365 < curr_year + max_day/365)].tolist()
    playerB_rows = df.index[(df['PlayerB_name'] == name) & (df['Year'] + df['Day']/365 < curr_year + max_day/365)].tolist()
    playerA_df = df.iloc[playerA_rows, playerA_cols]
    playerA_df['Win'] = 1
    playerB_df = df.iloc[playerB_rows, playerB_cols]
    playerB_df['Win'] = 0
    playerB_df.columns = list(playerA_df)
    tmp_df = pd.concat([playerA_df, playerB_df], ignore_index=True)
    if tmp_df.empty:
        continue
    
    # Compute a weight for each past match of the player
    tmp_df['elapsing_time'] = (curr_year + max_day/365) - (tmp_df['Year'] + tmp_df['Day']/365)
    tmp_df['weight'] = tmp_df['elapsing_time'].apply(lambda t: 0.8**t)
    tmp_df.loc[tmp_df['elapsing_time'] <= 1, 'weight'] = 1
    tmp_df['weight'] = (0.95 * tmp_df['weight']) + (0.05 * (weight_carpet*tmp_df['surface_Carpet'] + weight_clay*tmp_df['surface_Clay'] + weight_grass*tmp_df['surface_Grass'] + weight_hard*tmp_df['surface_Hard']))
    tmp_df.drop(columns=['Year', 'Day', 'elapsing_time', 'surface_Carpet', 'surface_Clay', 'surface_Grass', 'surface_Hard'], inplace = True)

    # Compute the weighted average of the player
    weighted_means = np.average(tmp_df, weights=tmp_df['weight'],axis=0)
    weighted_df = pd.DataFrame(weighted_means.reshape(-1, len(weighted_means)), columns=list(tmp_df.columns))
    weighted_df = weighted_df.drop('weight', axis=1)
    
    # Add stats of the player in new dataframe
    new_df.at[i, 8:] = weighted_df.iloc[0, 2:10]
    new_df.at[i, 'PlayerA_bestof'] = weighted_df['best_of']
    new_df.at[i, 'PlayerA_minutes'] = weighted_df['minutes']
    new_df.at[i, 'PlayerA_Win%'] = weighted_df['Win']
    
# Concat and updating columns names
new_df = pd.concat([players_df, new_df], axis=1)
column_names = [s[8:] for s in list(new_df.columns)]
new_df.columns = column_names

# Fill last missing values by median
new_df.fillna(new_df.median(), inplace=True)
new_df

,Name,FR,righthanded,age,rank,rank_points,Win%,bestof,minutes,svpt%,1st_serve%,1st_serve_won%,2nd_serve_won%,ace%,df%,bp_faced%,bp_saved%
1,Rafael Nadal,0.0,0.0,31.945244,2.0,7950.0,0.843530,3.533178,115.399078,0.472029,0.683603,0.733107,0.590587,0.038809,0.019196,0.060117,0.584363
2,Alexander Zverev,0.0,1.0,21.065023,3.0,6015.0,0.705283,3.240933,105.900699,0.506290,0.635341,0.736098,0.535599,0.082296,0.036593,0.062764,0.553850
3,Marin Cilic,0.0,1.0,29.623545,5.0,4770.0,0.674022,3.470694,112.005813,0.499401,0.568455,0.780501,0.545093,0.103401,0.029462,0.056462,0.563451
4,Grigor Dimitrov,0.0,1.0,26.995209,4.0,4870.0,0.635912,3.377729,110.328828,0.504455,0.624108,0.755760,0.522540,0.078886,0.038157,0.062971,0.573239
5,Juan Martin Del Potro,0.0,1.0,29.637236,6.0,4540.0,0.736424,3.401925,114.367182,0.490103,0.642172,0.759944,0.539136,0.089785,0.025161,0.053396,0.550948
6,Kevin Anderson,0.0,1.0,31.989048,7.0,3660.0,0.587122,3.386414,114.563273,0.513288,0.639217,0.771618,0.543078,0.131922,0.030039,0.052866,0.567695
7,Dominic Thiem,0.0,1.0,24.711842,8.0,3195.0,0.662268,3.391457,107.890212,0.492706,0.592241,0.746734,0.531827,0.068247,0.034668,0.067242,0.556411
8,David Goffin,0.0,1.0,27.433266,10.0,2930.0,0.621952,3.336416,106.828474,0.490723,0.584874,0.721764,0.520044,0.062512,0.035693,0.077393,0.562856
9,John Isner,0.0,1.0,33.067760,10.0,2955.0,0.621613,3.348492,113.911006,0.521751,0.690760,0.797883,0.563787,0.170943,0.020008,0.034405,0.546863
10,Pablo Carreno Busta,0.0,1.0,26.839151,11.0,2280.0,0.560846,3.329831,105.058510,0.492601,0.638428,0.691194,0.522927,0.045471,0.022315,0.081644,0.542788


In [107]:
new_df.to_csv('_Data/Predictions/stats_players_2018.csv', sep=',', encoding='utf-8', float_format='%.10f', decimal='.')